# Eumap Tutorial

![TMWM](../../img/change_sign.png "Change ahead")

Please access this using the new location:
* **GitLab**: https://gitlab.com/geoharmonizer_inea/eumap/-/blob/master/docs/notebooks/04_landcover_mapping_advanced.ipynb
* **Documentation**: https://eumap.readthedocs.org/notebooks/04_landcover_mapping_advanced.html